In [1]:
!export HF_HOME=../hf_cache

In [1]:
!pip install python-box
!pip install accelerate
!pip install datasets
!git config --global credential.helper store
!git config --global user.name "Neelectric"
!git config --global user.email "Neel.R@web.de"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 32.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 44.1 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstall

In [3]:
# import wandb
# from wandb import AlertLevel
# wandb.init(project="biollama_v2", # the project I am working on
#            tags=["hf_sft", "BioLlamaV2"]) # the Hyperparameters I want to keep track of

# wandb.alert(
#     title="Initialising training run",
#     text=f"We have started training",
#     level=AlertLevel.WARN,
#     wait_duration=300,
# )

In [4]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")
print(transformers.__version__)

There was a problem when trying to write in your cache folder (/root/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


imported...
4.36.2


Next, we pull the huggingface token from the local config. Remember to make a copy of config_default.yml, name it config.yml and add your huggingface token. config.yml is in the gitignore so unless you misspell config.yml, it should not get pushed to github.

In [5]:
# token = input(f"Enter token: ")
from box import Box
with open("config.yml", "r") as f:
    config = Box.from_yaml(f.read())
token = config.secrets.hf_token

We also need to verfiy whether the GPU supports bfloat16 (less precision but more range). If not (for example if running on TITAN GPUs), we use float16

In [6]:
if torch.cuda.is_bf16_supported():
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16
torch_dtype = torch.float32
print(f"Using dtype: {torch_dtype}")

Using dtype: torch.float32


In [7]:
print("started script")

# llama_path = "h2oai/h2ogpt-4096-llama2-7b-chat"
llama_path = "meta-llama/Llama-2-7b-chat-hf"
# llama_path = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(llama_path, 
                                          token = token,
                                          cache_dir = "../hf_cache/")
model = AutoModelForCausalLM.from_pretrained(llama_path, 
                                             token = token,
                                            #  device_map = "auto",
                                            device_map = "cuda:0",
                                             torch_dtype = torch_dtype,
                                             cache_dir = "../hf_cache/")

print("model loaded!")

started script


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded!


In [8]:
from time import time
max_new_tokens = 150
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda:0')
time_before = time()
raw_output = model.generate(tokenized_prompt,
                            max_new_tokens = max_new_tokens,
                            temperature = 0.01)
time_after = time()
time_taken = time_after - time_before
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
num_generated = len(raw_output[0]) - len(tokenized_prompt[0])
print(untokenized_output)
print(f"newly generated {num_generated}")
print(f"{num_generated / time_taken} t/s")


Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
(D) Ketogenesis

Explanation:
A low insulin to glucagon ratio is typically seen in states of ketosis, such as during fasting or in the presence of high-fat diets. Glucagon is a hormone produced by the pancreas that raises blood glucose levels by stimulating the breakdown of glycogen and fatty acids. Insulin, on the other hand, lowers blood glucose levels by promoting the uptake of glucose by cells. In ketosis, the body shifts from relying primarily on glucose for energy to relying on ketones, which are produced
newly generated 150
11.760068695805336 t/s


In [9]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

for param in model.parameters(): 
    param.requires_grad = True
model.model.embed_tokens.weight.requires_grad_(False)
params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 6607.34M


In [10]:
import os
from tqdm import tqdm
from datasets import Dataset
from datasets import load_dataset
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/pubmed_cleaned/"
print(path)
files = os.listdir(path)
print(files[:5])
# load in the first file and print first 20 lines
# encoded_dataset = []
# with open(path + files[0], "r") as f:
#     lines = f.readlines()
#     for line in tqdm(lines):
#         encoded_block = tokenizer.encode(line)
#         encoded_dataset.append(encoded_block)
# train_data = Dataset.from_list(encoded_dataset)
print(path + files[0])
dataset = load_dataset("text", 
                       cache_dir = "../hf_cache/",
                       data_files = path + files[0], 
                    #    split = "train"
                       )

/nfs/primary/pubmed_cleaned/
['abs_1_14.tsv', 'abs_1_15.tsv', 'abs_1_23.tsv', 'abs_1_0.tsv', 'abs_1_30.tsv']
/nfs/primary/pubmed_cleaned/abs_1_14.tsv


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 67773
    })
})

In [12]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=2048, return_tensors="pt"), batched=True)

Map:   0%|          | 0/67773 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
print(data_collator)

DataCollatorForLanguageModeling(tokenizer=LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')


In [16]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 67773
    })
})


In [17]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="/root/nfs/trained_models",
    evaluation_strategy="no",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    max_steps=1000,
    warmup_steps=100,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    report_to=[],
)

In [18]:
type(dataset["train"][0]["input_ids"])

list

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=data_collator,
)

trainer.train()

/opt/miniconda3/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
batch_size = 2
total_num_steps = 500000
output_dir = "../"
training_args = TrainingArguments(
    output_dir=output_dir,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size//2,
    # bf16=False,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=total_num_steps // 10,
    num_train_epochs=1,
    # max_steps = -1
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    # evaluation_strategy="steps",
    # eval_steps=total_num_steps // 6,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="epoch", #changed to epoch so we save every epoch i guess?
    save_total_limit=1,
)

In [16]:
def create_prompt(row):
    text = row["text"]
    return text
# create_prompt(dataset[0])

In [17]:
train_dataset = dataset["text"]

In [18]:
# print(train_dataset[0])

In [19]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    # eval_dataset=test_dataset,
    packing=True,
    max_seq_length=512,
    args=training_args,
    formatting_func=create_prompt,
    # compute_metrics=token_accuracy,
)

In [20]:
#set llama model config use_cache to false!!!
trainer.train()
# wandb.finish()

In [21]:
# trainer.save_model(output_dir)
# #print contents of output_dir
# !ls -l $output_dir
# #print full path of output_dir
# # !pwd $output_dir

In [22]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class TextDataset(Dataset):
#     def __init__(self, data, tokenizer):
#         self.data = data
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         text = self.data[idx]['text']
#         inputs = self.tokenizer(text, 
#                                 return_tensors="pt", 
#                                 padding=True, 
#                                 max_length=10,
#                                 truncation=True)
#         return inputs
# text_dataset = TextDataset(dataset, tokenizer)
# dataloader = DataLoader(text_dataset, batch_size=1, shuffle=True)

In [23]:
# import torch.optim as optim

# def train(model, dataloader, epochs=5, lr=5e-5):
#     device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#     # model.to(device)

#     optimizer = optim.AdamW(model.parameters(), lr=lr)
#     print("starting training")

#     for epoch in range(epochs):
#         model.train()
#         for item in dataset:
#             print(len(item))
#             text = item["text"]
#             inputs = tokenizer(text, return_tensors="pt", padding=True, max_length=10, truncation=True)
#             # print(batch)
#             inputs = inputs.to(device)
#             print(inputs.input_ids.shape)
#             print(inputs)
#             outputs = model(**inputs, labels=inputs.input_ids)
#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()

#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [24]:
# dataset[0]["text"]

'\tTwenty male patients with primary hypertriglyceridemia were treated for 4 weeks with daily supplements (15 g) of oil, which provided approximately 6 g of polyunsaturated fatty acid (PUFA) either of fish or of vegetable origin. Total plasma cholesterol concentrations were unaffected, but both types of supplement increased high density lipoprotein-3 (HDL3) cholesterol concentrations. The fish, but not the vegetable, oil supplement led to a decrease in plasma triglyceride concentrations. Very low density lipoprotein (VLDL), fatty acid composition, and VLDL triglyceride kinetics were subsequently studied in five patients (four male, one female) before and after 4 weeks of therapy with 15 g of the same fish oil. The fish oil led to increases in the proportion of eicosapentaenoic acid in both the VLDL triglyceride and phospholipid fractions, but the increase was greater in the latter. In contrast, the proportion of docosahexanoic acid was increased only in the VLDL triglycerides. The decr

In [25]:
# tokenizer.pad_token = tokenizer.eos_token
# train(model, "dataloader", epochs=5, lr=5e-5)

starting training
1
torch.Size([1, 10])
{'input_ids': tensor([[    1, 29871,    12, 27418,  6478, 14263, 22069,   411,  7601, 11266]],
       device='cuda:1'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:1')}


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 1 has a total capacity of 47.54 GiB of which 70.62 MiB is free. Process 18613 has 16.10 GiB memory in use. Process 109231 has 31.37 GiB memory in use. Of the allocated memory 30.14 GiB is allocated by PyTorch, and 60.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 